# Jonathan Halverson
# Monday, December 26, 2016
# Creating a schema

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Schema").getOrCreate()

In [2]:
type(spark)

pyspark.sql.session.SparkSession

In [3]:
cars = spark.read.csv('cars.dat', header=False, inferSchema=False)
cars.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



In [4]:
cars = spark.read.csv('cars.dat', header=False, inferSchema=True)
cars.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: string (nullable = true)



In [5]:
cars.show()

+-----+------+-----+---+------+---+
|  _c0|   _c1|  _c2|_c3|   _c4|_c5|
+-----+------+-----+---+------+---+
|Sedan| Camry|31000|  2| 65.91|  D|
|Sedan| Civic|27000|  1| 99.05|  D|
|Truck|  F150|40500|  3|1100.0|  E|
|Truck|Ranger|32000|  3| 950.0|  E|
+-----+------+-----+---+------+---+



In [9]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, DoubleType, IntegerType
schema_ = StructType([StructField("Vehicle", StringType(), True),
                      StructField("Model", StringType(), True),
                      StructField("Cost", IntegerType(), True),
                      StructField("Type", IntegerType(), True),
                      StructField("Labor", DoubleType(), True),
                      StructField("Class", StringType(), True)])

In [10]:
cars = spark.read.csv('cars.dat', schema=schema_, header=False, inferSchema=False)
cars.printSchema()

root
 |-- Vehicle: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Cost: integer (nullable = true)
 |-- Type: integer (nullable = true)
 |-- Labor: double (nullable = true)
 |-- Class: string (nullable = true)



In [11]:
cars.show()

+-------+------+-----+----+------+-----+
|Vehicle| Model| Cost|Type| Labor|Class|
+-------+------+-----+----+------+-----+
|  Sedan| Camry|31000|   2| 65.91|    D|
|  Sedan| Civic|27000|   1| 99.05|    D|
|  Truck|  F150|40500|   3|1100.0|    E|
|  Truck|Ranger|32000|   3| 950.0|    E|
+-------+------+-----+----+------+-----+

